In [1]:
var {
    read,
    fetchData,
    getSummarizedParameters,
    groupEachStage,
    palette,
    integratedLiquidLine,
    Plotly
} = require("./show-magma-model")

var funcTools = require("../../jslib/funcTools")

var fontMaker = (fontSize) => {
    return {
        family: 'sans-serif',
        size: fontSize,
        color: '#000'
    }
}

var middiumSize = {
    width : 800,
    height : 600
}

var largeSize = {
    width : 1000,
    height : 750
}

var legend= (normalOrder = true) => {
    return {
        legend: {
            traceorder : (normalOrder)? "normal" : "reversed" ,
            font: fontMaker(20)
        }
    }
    
}


var spread = (...objs) => [...objs].reduce((a,b)=>Object.assign(a,b),{})


var axis = (direction,title,range) => {
    const obj = {}
    obj[direction+"axis"] = {
        title : title, 
        titlefont:fontMaker(24),
        tickfont : fontMaker(18)
    };
    if (range !== undefined) obj[direction+"axis"].range = range;
    return obj
}

Directory

```
/diffusion
/phase
/ptmcmc
    /model
    /tool
    /result
    
```

In [61]:
var resultPath = "../result/1002-9-1C-opx1/2018-0719/"
var dataPath = "../model/"

var lavaPath = "../../phase/data/lava_compositions_mafic_NEshikoku.csv"
var opxPath = "../../phase/data/opx_zoning_compile.csv"

In [62]:
var {
    meta,
    data,
    error,
    option,
    summary
} = fetchData(resultPath,dataPath)

var {
      model,
      parameters,
      updateCondition,
      constrain,
      magma
    } = require("../"+meta.model)(option)

var cmap = palette("tol-rainbow", parameters.length)

var lava = read.csv(lavaPath)
var opx = read.csv(opxPath)


Summary column [ '', 'parameter', 'mean', 'stdev', 'mode', 'max_P' ]


In [65]:
var summarizedParameters = getSummarizedParameters(summary, "mean", parameters);
var parametersEachStage = groupEachStage(summarizedParameters)

summarizedParameters[0]

{ MgN_beforeCrystallization: '92.28146176775996',
  growth_stoichiometry_orthopyroxene: '0.25771838565511407',
  growth_stoichiometry_spinel: '0.04545101595541584',
  MgN_beforeMixing: '91.80840668119',
  mixing_stoichiometry_orthopyroxene: '0.38382499411106985',
  mixing_stoichiometry_spinel: '0.04749268167923583',
  log10_tau: '8.047985619172266' }

In [48]:
Plotly([
    {
        y : funcTools.zipWith((x,y)=>1-x-y)
            (parametersEachStage.growth_stoichiometry_orthopyroxene)
            (parametersEachStage.growth_stoichiometry_spinel),
        name : "Olivine",
        type:"bar"
    },
    {
        y : parametersEachStage.growth_stoichiometry_orthopyroxene,
        name : "Orthopyroxene",
        type:"bar"
    },
    {
        y : parametersEachStage.growth_stoichiometry_spinel,
        name : "Spinel",
        type:"bar"
    }
],
spread(
    middiumSize,
    legend(),
    axis("x", "Stage number"),
    axis("y", "Stoichiometry", [0,1]),
    {
        barmode: "stack",
        title : "Stoichiometry in crystal growth"
    }
))

In [49]:
Plotly([
    {
        y : funcTools.zipWith((x,y)=>1-x-y)
            (parametersEachStage.mixing_stoichiometry_orthopyroxene)
            (parametersEachStage.mixing_stoichiometry_spinel),
        name : "Olivine",
        type:"bar"
    },
    {
        y : parametersEachStage.mixing_stoichiometry_orthopyroxene,
        name : "Orthopyroxene",
        type : "bar"
    },
    {
        y : parametersEachStage.mixing_stoichiometry_spinel,
        name : "Spinel",
        type : "bar"
    }
],
spread(
    middiumSize,
    legend(),
    axis("x", "Stage number"),
    axis("y", "Mass fraction", [0,1]),
    {
        barmode: "stack",
        title : "Mass fraction in approximating mixing",
    }
)
)


In [50]:
Plotly([
    {
        y : parametersEachStage.log10_tau
    }
],
spread(
    middiumSize,
    legend(),
    axis("x", "Stage number"),
    axis("y", "log10 net time scale of diffusion"),
    {
        title : ""
    }
)
)

In [51]:
var modeled = model(summarizedParameters,data)
var ndProfiles = magma.diffusionProfiles.orthopyroxene;

var ndFe_Mg = ndProfiles.Fe_Mg.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Fe_Mg.map(v=>100/(1+v)),
        name : "Model: diffused"
    },
     {
        x : ndFe_Mg.x,
        y : ndFe_Mg.c.map(v=>100/(1+v)),
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Fe_Mg.map(v=>100/(1+parseFloat(v))),
        mode : "markers",
        name : "Observed"
    }

],
spread(
    middiumSize,
    legend(),
    axis("x","Radius"),
    axis("y","Mg#")
)
)

In [52]:
var modeled = model(summarizedParameters,data)
var ndProfiles = magma.diffusionProfiles.orthopyroxene;

var ndFe_Mg = ndProfiles.Fe_Mg.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Fe_Mg,
        name : "Model: diffused"
    },
     {
        x : ndFe_Mg.x,
        y : ndFe_Mg.c,
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Fe_Mg,
        mode : "markers",
        name : "Observed"
    }

],
spread(
    middiumSize,
    legend(),
    axis("x","Radius"),
    axis("y","Fe/Mg")
))

In [53]:
var ndCr2O3 = ndProfiles.Cr2O3.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Cr2O3,
        name : "Model: diffused"
    },
    {
        x : ndCr2O3.x,
        y : ndCr2O3.c,
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Cr2O3,
        mode : "markers",
        name : "Observed"
    }

],
spread(
    middiumSize,
    legend(),
    axis("x","Radius"),
    axis("y","Cr2O3")
))

In [73]:
Plotly(
    [
        {
            x : opx.Fe_Mg.map(v=>100/(1+parseFloat(v))),
            y : opx.Cr2O3,
            name : "Opx",
            mode : "markers",
            marker : {
                color : "black",
                size : 2
            }
        },
        {
            x : modeled.Fe_Mg.map(v=>100/(1+v)),
            y : modeled.Cr2O3,
            name : "Model: diffused"
        },
        /*{
            x : ndFe_Mg.c.map(v=>100/(1+v)),
            y : ndCr2O3.c,
            name : "Model: original"
        },*/
    ],
    spread(
        middiumSize,
        axis("x","Mg#",[60,95]),
        axis("y","Cr2O3",[0,0.5])
    )
)

In [54]:

var liquidLine = integratedLiquidLine(magma,cmap)

//console.log(liquidLine("SiO2","MgO"))


In [55]:

Plotly(
    [
        ...liquidLine("SiO2","MgO"),
        {
            x : lava.SiO2,
            y : lava.MgO,
            mode : "markers",
            marker: {
                color : "black"
            },
            name : "Lavas"
        }
    ],
    spread(
        largeSize,
        legend(false),
        axis("x","SiO2"),
        axis("y","MgO")
    )      
)


In [56]:
Plotly(
    [
        ...liquidLine("SiO2","Cr2O3"),
        {
            x : lava.SiO2,
            y : lava.Cr2O3,
            mode : "markers",
            marker: {
                color : "black"
            },
            name : "Lavas"
        }
    ],
    spread(
        largeSize,
        legend(false),
        axis("x","SiO2"),
        axis("y","Cr2O3")
    )
)


In [57]:

Plotly(liquidLine("SiO2","Al2O3"),
spread(
    largeSize,
    legend(false),
    axis("x","SiO2"),
    axis("y","Al2O3")
)      
)